# Uniswap Decentralized Exchange Pool Analysis

Owner: April Nellis

Companion code to [*DEX Specs: A Mean-Field Approach to DeFi Cryptocurrency Exchanges*](https://arxiv.org/abs/2404.09090)

In [ ]:
import numpy as np
import pandas as pd
import math
import itertools
pd.options.mode.chained_assignment = None  # default='warn'
import os.path
from datetime import datetime, timedelta
import time
import requests
import re
import eth_abi
import statsmodels.api as sm
import scipy.stats as stats
from scipy.optimize import curve_fit
from scipy.stats import wasserstein_distance
from scipy import optimize
from IPython.display import Image
from coinmetrics.api_client import CoinMetricsClient
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import import_ipynb
from IPython.display import IFrame
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.neighbors import KernelDensity
from decimal import Decimal

from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Just disables the annoying warning, doesn't enable AVX/FMA
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

%matplotlib widget

#plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = [7,5]

## Etherscan Data Analysis

### Initializing pool information

Most analysis is performed on September 1-30, 2023 data. You may change the `start_date` and `end_date` variables to alter the blocks investigated.

In [ ]:
start_date = datetime(2023, 9, 1, 0, 0, 0) # year, month, day, hour, minute, second
end_date = datetime(2023, 9, 30, 0, 0, 0)

start_str = start_date.strftime('%Y-%m-%d')
start_unix = int(time.mktime(start_date.timetuple()))

end_str = end_date.strftime('%Y-%m-%d')
end_unix = int(time.mktime(end_date.timetuple()))

start_block = 18039179
end_block = 18245998

If you wish to change the dates, input the following API keys and run the code to find the correct start and end blocks:

In [ ]:
API_KEY = '' # Input your personal Etherscan API key here (free)

In [ ]:
GECKO_API = '' # Enter your personal CoinGecko API demo key here (free)

In [ ]:
my_url = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={start_unix}&closest=before&apikey={API_KEY}'

response = requests.get(my_url)
resp = response.json()
start_block = int(resp['result'])

my_url = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={end_unix}&closest=before&apikey={API_KEY}'

response = requests.get(my_url)
resp = response.json()
end_block = int(resp['result'])

print(f'The period from {start_date} to {end_date} encompasses blocks {start_block} to {end_block} ({end_block-start_block} blocks).')

Coins of Interest:
* **Ethereum (ETH)**
    * This coin fluctuates in value compared to the US dollar. Native token of the Ethereum blockchain.
* **(Wrapped) Bitcoin (WBTC)**
    * The Bitcoin blockchain does not support smart contracts, which do most of the work of a decentralized exchange. Therefore, Bitcoin must be "wrapped" to be used in decentralized finance. This means that a token representing a Bitcoin is used in its place when placing transactions on other blockchains like the Ethereum blockchain.
* **USD Coin (USDC)**
    * This is a stablecoin with a value of 1 USD. It is supposed to be 1:1 collateralized with US dollars. 
* **Tether (USDT)**
    * This is the most popular (as of Jan 2023) stablecoin, also having a value of 1 USD. It is supposed to be 1:1 collateralized with US dollars. 
* **Dai (DAI)**
    * This is an algorithmic stablecoin which is also intended to have a value of 1 USD. It is overcollateralized using Ethereum via a series of smart contracts with the MakerDAO organization.

In [ ]:
# Contract addresses for each coin:
ETH = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
WBTC = '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'
USDC = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
USDT = '0xdAC17F958D2ee523a2206206994597C13D831ec7'
DAI = '0x6B175474E89094C44Da98b954EedeAC495271d0F'

**Choose pool of interest from below options. Uncomment and run the cell corresponding to your choice to choose the pool index.**

[**DAI/USDC Pool with 0.05% Fee**](https://info.uniswap.org/#/pools/0x6c6bc977e13df9b0de53b251522280bb72383700) This pool allows exchanges of DAI and USDC. Activity in this pool has dropped off in favor of the DAI/USDC pool with a 0.01% fee, though both pools have more TVL and less trading volume than pairs with more volatility.

In [ ]:
#POOL = 1

[**DAI/USDC Pool with 0.01% Fee**](https://info.uniswap.org/#/pools/0x5777d92f208679db4b9778590fa3cab3ac9e2168) This pool allows exchanges of DAI and USDC. Activity in this pool is frequent but trade sizes are relatively small.

In [ ]:
#POOL = 2

[**USDC/ETH Pool with 0.05% Fee**](https://info.uniswap.org/#/pools/0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640): This pool facilitates swaps between USDC and Ethereum. This pool has the largest daily trading volume and a much higher TVL than other USDC/ETH pools (as of 01/24/2023).

In [ ]:
# This is the pool discussed in the paper
POOL = 3

[**USDC/ETH with 0.01% Fee**](https://info.uniswap.org/#/pools/0xe0554a476a092703abdb3ef35c80e0d76d32939f): This pool facilitates swaps between USDC and Ethereum. This pool did not have much activity until November 17-18, 2022, after which its use has been steading growing. The pool may not have existed before this time - further investigation necessary. 

In [ ]:
#POOL = 4

[**USDC/USDT with 0.01% Fee**](https://info.uniswap.org/#/pools/0x3416cf6c708da44db2624d63ea0aaef7113527c6): This pool facilitates swaps between USDC and Tether. This stablecoin pair sees more activity than the USDC/DAI pairing, perhaps because Tether is preferred to DAI in certain cryptocurrency uses or because Tether is viewed as more reliable.

In [ ]:
#POOL = 5

[**WTBC/ETH with 0.05% Fee**](https://info.uniswap.org/#/pools/0x4585fe77225b41b697c938b018e2ac67ac5a20c0): This pool facilitates swaps between wrapped Bitcoin and Ethereum. Note that the prices of both these token experience high volatility.

In [ ]:
#POOL = 6

[**ETH/USDT with 0.05% Fee**](https://info.uniswap.org/#/pools/0x11b815efb8f581194ae79006d24e0d814b7697f6): This pool facilitates swaps between Ethereum and Tether. While USDC is still the preferred stablecoin to swap with Ethereum, this pool also has a decent amount of activity. However, trading volume appears to have decreased significantly since May 2022. 

In [ ]:
#POOL = 7

**Define the associated parameters.**

`POOL` is the key, and the value is a tuple storing `(TOK_A, TOK_A_ADR, TOK_B, TOK_B_ADR, FEE, PROTOCOL_STR, PROTOCOL)`

In [ ]:
wallet = {1:('DAI', DAI, 'USDC', USDC, '005', 'Uniswap V3: DAI-USDC', '0x6c6bc977e13df9b0de53b251522280bb72383700'),
         2:('DAI', DAI, 'USDC', USDC, '001', 'Uniswap V3: DAI-USDC 4', '0x5777d92f208679db4b9778590fa3cab3ac9e2168'),
         3:('ETH', ETH, 'USDC', USDC,'005', 'Uniswap V3: USDC 3', '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'),
         4:('ETH', ETH, 'USDC', USDC,  '001', 'Uniswap V3: USDC 4', '0xe0554a476a092703abdb3ef35c80e0d76d32939f'),
         5:('USDC', USDC, 'USDT', USDT,'001','Uniswap V3: USDC-USDT 4','0x3416cf6c708da44db2624d63ea0aaef7113527c6'),
         6:('WBTC', WBTC, 'ETH', ETH, '005', 'Uniswap V3: WBTC 2', '0x4585fe77225b41b697c938b018e2ac67ac5a20c0'),
         7: ('ETH', ETH, 'USDT', USDT, '005', 'Uniswap V3: USDT 3', '0x11b815efb8f581194ae79006d24e0d814b7697f6')}

Load pool-specific information

In [ ]:
(TOK_A, TOK_A_ADR, TOK_B, TOK_B_ADR, FEE, PROTOCOL_STR, PROTOCOL) = wallet[POOL]

### Notable user information

In [ ]:
# Identifying notable users
weird_bot_1 = '0xa69babef1ca67a37ffaf7a485dfff3382056e78c' # Only does one half of transaction, always through Uniswap router
weird_bot_2 = '0xe841e62778d997729cbbe165b029ebb4af8a58ab' # Paired with weird_bot_1, address not labeled as MEV but txn labeled as MEV

In [ ]:
NFT = '0xc36442b4a4522e871399cd717abdd847ab11fe88' # Uniswap V3: Positions NFT (tracks liquidity position)
ROUTER = '0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45|0xe592427a0aece92de3edee1f18e0157c05861564|0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b' # Uniswap V3: Router 2, facilitates exchanges

In [ ]:
bots = []
bots.append('0x57c1e0c2adf6eecdb135bcf9ec5f23b319be2c94')
bots.append('0x00000000032962b51589768828ad878876299e14')
bots.append('0xe8c060f8052e07423f71d445277c61ac5138a2e5')
bots.append('0x83bc685ebd7e641f83f45cecdfe62b87afaef9c7') # Associated with 0xe8c06.. bot, same creator
bots.append('0x00000000003b3cc22af3ae1eac0440bcee416b40')
bots.append('0x493f461aead031cee2027f1b95370a692611acb9')
bots.append('0x5050e08626c499411b5d0e0b5af0e83d3fd82edf')
bots.append(weird_bot_1)
bots.append('0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf')
bots.append(weird_bot_2)
bots.append('0x56178a0d5f301baf6cf3e1cd53d9863437345bf9')

MEV = bots[0] #MEV bots, labeled by Etherscan - maximal extractable value
for bot in bots[1:]:
    MEV = MEV + '|' + bot

### Loading and Cleaning Data

#### Function definitions
ATTENTION! This section only needs to be run if you change the dates or the pool in the initialization.

The function `get_dataframe` takes a token name string (`tok_name`), the corresponding token address (`tok_adr`), the address associated with the Uniswap protocol (`protocol`), and the starting and ending block numbers (`start` and `end`). Note the addresses are long strings. It returns a Pandas Dataframe containing the first 10,000 transactions within the given block range, involving the specified token and protocol.

In [ ]:
def get_dataframe(tok_name, tok_adr, protocol, start, end):
    start = int(start)
    end = int(end)
    
    my_url = f"https://api.etherscan.io/api?module=account&action=tokentx&contractaddress={tok_adr}&address={protocol}&startblock={start}&endblock={end}&sort=asc&apikey={API_KEY}"
    response = requests.get(my_url)
    resp = response.json()
    if int(resp['status']) == 1:
        df = pd.DataFrame(resp['result'])
        df['blockNumber'] = df['blockNumber'].astype(int)
        df['value'] = df['value'].astype(float)/np.power(10, df['tokenDecimal'].astype(int))
        df['gasPrice'] = df['gasPrice'].astype(float)/(1e18) # gas price given in ETH, adjusted by 18 decimal places
        df['gasFee']=df['gasPrice']*(df['gasUsed'].astype(float))

        df.rename(columns={'from':f'From{tok_name}', 'to':f'To{tok_name}','value':tok_name, 'hash':'Txnhash'}, inplace = True)
        df.drop(['nonce', 'tokenName', 'tokenSymbol', 'transactionIndex', 'tokenDecimal', 'input', 'confirmations', 'gas', 'cumulativeGasUsed'], axis = 1, inplace = True)

        if (df['contractAddress'] == tok_adr.lower()).all():
            df.drop(['contractAddress'], axis = 1, inplace = True)
        else:
            print('Address mismatch?')
    else:
        df = pd.DataFrame(columns = ['blockNumber', 'timeStamp', 'Txnhash', 'blockHash', f'From{tok_name}', f'To{tok_name}', tok_name, 'gasPrice', 'gasUsed', 'gasFee'])
    return df

In [ ]:
# Example function usage:
temp = get_dataframe(TOK_A, TOK_A_ADR, PROTOCOL, start_block, start_block + 3)
if temp.empty:
    print('Empty')
temp

Get the logs and information for a specific transaction hash.

In [ ]:
def get_txn(txnhash):
    txn_url=f'https://api.etherscan.io/api?module=proxy&action=eth_getTransactionReceipt&txhash={txnhash}&apikey={API_KEY}'
    response = requests.get(txn_url)
    resp = response.json()
    from_usr = resp['result']['from']
    to_usr = resp['result']['to']
    
    return (from_usr, to_usr)

In [ ]:
# Example function usage
(a, b) = get_txn('0x11a6cfccf1f3684c68c6086be88ce16fce742fe8068f32bd9b976b33589c8968')
print(f"This transaction is from {a} to {b}.")

Identify users who were obscured by the Uniswap router for both the incoming and outgoing transactions. If the master file is already saved as a CSV, this has already been taken care of. 

In [ ]:
# This takes a long time if there is a long list of transactions because it requires a lot of API queries
def replace_router(df):
    a = df[f'User{TOK_A}'].str.contains(ROUTER)
    b = df[f'User{TOK_B}'].str.contains(ROUTER)
    txn_list = df['Txnhash'][a*b] # both users are the router (if only one, we already have some info)
    print(f'There are {np.sum(a*b)} router transactions.')
    
    for idx in txn_list.index:
        txn = df['Txnhash'][idx]
        (from_usr, to_usr) = get_txn(txn)

        if to_usr not in ROUTER:
            print(f'Recipient is {to_usr} at index {idx}.')

        if df[TOK_A].iloc[idx] > 0:
            df[f'User{TOK_A}'].iloc[idx] = from_usr
            df[f'User{TOK_B}'].iloc[idx] = to_usr
        else:
            df[f'User{TOK_B}'].iloc[idx] = from_usr
            df[f'User{TOK_A}'].iloc[idx] = to_usr
        
    return df

Identify users who are obscured by the Positions NFT contract when withdrawing their liquidity.

In [ ]:
def replace_nft(df):
    a = df[f'User{TOK_A}'].str.contains(NFT)
    b = df[f'User{TOK_B}'].str.contains(NFT)
    txn_list = df.index[a*b] # both users are the Positions NFT
    print(f'There are {np.sum(a*b)} Positions NFT transactions.')
    
    for idx in txn_list:
        txn = df['Txnhash'].loc[idx]
        (from_usr, to_usr) = get_txn(txn)
        #print(from_usr, to_usr)

        if to_usr == NFT and from_usr == NFT:
            print('ugh, no new info')
        elif from_usr != NFT:
            df[f'User{TOK_A}'].loc[idx] = from_usr # leave this one for now
            #df[f'User{TOK_B}'].loc[idx] = from_usr 
        elif to_usr != NFT:
            print(f'weird at {idx}')
        else:
            print(f'Extra user {to_usr} at {idx}.')
            df[f'User{TOK_A}'].loc[idx] = from_usr
            #df[f'User{TOK_B}'].loc[idx] = from_usr
        
    return df

Since CoinMetrics hourly data is restricted to paid users and my free trial expired, we switch to CoinGecko, which has hourly data available for any date in the last 90 days. Granularity is automatic, so if the dates are more than 90 days in the past, daily data will be provided instead (alas).

In [ ]:
def get_gecko_prices(token):
    if token == 'ETH':
        coin_id = 'ethereum'
    elif token == 'WBTC':
        coin_id = 'bitcoin'
    elif token == 'USDC':
        coin_id = 'usd-coin'
    elif token == 'USDT':
        coin_id = 'tether'
    elif token == 'DAI':
        coin_id = 'dai'
    else:
        coin_id = ''
        
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range?vs_currency=usd&from={start_unix}&to={end_unix}&precision=full'
    url_key = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range?vs_currency=usd&from={start_unix}&to={end_unix}&precision=full&x_cg_demo_api_key={GECKO_API}'
    response = requests.get(url)
    resp = response.json()
    if 'error' in resp.keys():
        print(resp)
    else:
        price_df = pd.DataFrame(resp["prices"])
        price_df.rename(columns={0: "time", 1: "price"}, inplace = True)
        price_df["time"] = pd.to_datetime(price_df["time"], unit='ms')
        return price_df

In [ ]:
get_gecko_prices(TOK_B)

#### Create the master DataFrame
ATTENTION! This section only needs to be run if you change the dates or the pool in the initialization.

Pull data from Etherscan if necessary

In [ ]:
# TOKEN A
fileStringA = f'data/pool{POOL}_{TOK_A}_{start_block}_{end_block}.csv'
fileExistsA = os.path.isfile(fileStringA)

if fileExistsA:
    print('Loading CSV file...')
    token_a = pd.read_csv(fileStringA)
    token_a.drop('Unnamed: 0', axis = 1, inplace = True)
    print('Done.')
else:
    print('Pulling data from Etherscan...')
    token_a = get_dataframe(TOK_A, TOK_A_ADR, PROTOCOL, start_block, end_block)
    start_block_temp = token_a['blockNumber'].iloc[-1]
    
    while start_block_temp < end_block:
        # start from the next block with next 10000 transactions
        print(f'Block {start_block_temp}')
        temp = get_dataframe(TOK_A, TOK_A_ADR, PROTOCOL, start_block_temp, end_block)
        
        if temp.empty:
            break
        else:
            token_a = pd.concat([token_a, temp], ignore_index = True)

            start_block_temp = token_a['blockNumber'].iloc[-1]

            # fill in the potential additional transactions in last block within batch of 10000 transactions
            temp = get_dataframe(TOK_A, TOK_A_ADR, PROTOCOL, start_block_temp, start_block_temp)
            token_a = pd.concat([token_a, temp], ignore_index = True)  
            start_block_temp += 1

            time.sleep(0.5) # just to ensure that the speed does not exceed the 5 request/second limit for a free account
    
    token_a.to_csv(fileStringA)
    print('Done.')

In [ ]:
# TOKEN B
fileStringB = f'data/pool{POOL}_{TOK_B}_{start_block}_{end_block}.csv'
fileExistsB = os.path.isfile(fileStringB)

if fileExistsB:
    print('Loading CSV file...')
    token_b = pd.read_csv(fileStringB)
    token_b.drop('Unnamed: 0', axis = 1, inplace = True)
    print('Done.')
else:
    print('Pulling data from Etherscan...')
    token_b = get_dataframe(TOK_B, TOK_B_ADR, PROTOCOL, start_block, end_block)
    start_block_temp = token_b['blockNumber'].iloc[-1]
    
    while start_block_temp < end_block:
        # start from the next block with next 10000 transactions
        print(f'Block {start_block_temp}')
        temp = get_dataframe(TOK_B, TOK_B_ADR, PROTOCOL, start_block_temp, end_block)
        if temp.empty:
            break
        else:
            token_b = pd.concat([token_b, temp], ignore_index = True)

            start_block_temp = token_b['blockNumber'].iloc[-1]

            # fill in the potential additional transactions in last block within batch of 10000 transactions
            temp = get_dataframe(TOK_B, TOK_B_ADR, PROTOCOL, start_block_temp, start_block_temp)
            token_b = pd.concat([token_b, temp], ignore_index = True)  
            start_block_temp += 1
             
            time.sleep(0.5) # just to ensure that the speed does not exceed the 5 request/second limit for a free account
            
    token_b.to_csv(fileStringB)
    print('Done.')

Clean data and merge DataFrames

In [ ]:
new_a = token_a.drop_duplicates(subset='Txnhash', keep=False)
new_b = token_b.drop_duplicates(subset='Txnhash', keep=False)

if len(new_a) != len(new_b):
    print(f'MISMATCH: there are {len(new_a)} {TOK_A} transactions and {len(new_b)} {TOK_B} transactions.')

N = min(len(new_a), len(new_b))

In [ ]:
# Combine token A and token B transaction data based on transaction hashes
master = new_a.merge(new_b, how = 'inner', on = 'Txnhash')
if len(master) != N:
    print(f'LOST INFORMATION: there are {len(master)} merged rows vs {N} original transactions.')

In [ ]:
# Clean up column names and improve readability
for string in ['blockNumber', 'timeStamp', 'blockHash', 'gasFee', 'gasPrice','gasUsed']:
    if (master[f'{string}_x'] == master[f'{string}_y']).all():
        master.drop(f'{string}_y', axis = 1, inplace = True)
        master.rename(columns={f'{string}_x':string}, inplace = True)
    else:
        print(string)
        print(np.sum(master[f'{string}_x']== master[f'{string}_y']))

master.drop('blockHash', axis = 1, inplace = True)
master['timeStamp'] = pd.to_datetime(master['timeStamp'], unit = 's')

Calculate exchange rate without taking into acccount Uniswap transaction fee:

In [ ]:
# drop very small transactions as they add weird things
master = master[np.abs(master[TOK_B]) > 1]

In [ ]:
master['Exchange Rate'] = np.abs(master[TOK_B])/master[TOK_A]
master['Exchange Rate'].replace([np.inf, -np.inf], np.nan, inplace=True)

Categorize transactions as Add/Swap/Remove

In [ ]:
# Determine whether, for each transaction, money is entering or exiting Uniswap wallet 

a = -1*np.array(master[f'From{TOK_A}'].str.contains(PROTOCOL), dtype = 'int') # USDC leaving Uniswap
b = np.array(master[f'To{TOK_A}'].str.contains(PROTOCOL), dtype = 'int') # USDC entering Uniswap
uniswap_a = a+b # token A transactions from uniswap's perspective

c = -1*np.array(master[f'From{TOK_B}'].str.contains(PROTOCOL), dtype = 'int') # DAI leaving Uniswap
d = np.array(master[f'To{TOK_B}'].str.contains(PROTOCOL), dtype = 'int') #Dai entering Uniswap
uniswap_b = c+d # token B transactions from uniswap's perspective

# Check for txns where Uniwasp V3: TOK_A:TOK_B contract wallet not found
if 0 in uniswap_a:
    print(f"Weird {TOK_A} transaction!") 
if 0 in uniswap_b:
    print(f'Weird {TOK_B} transaction!')

# Create signed float transactions from Uniswap LP's viewpoint
master[TOK_A] = master[TOK_A]*uniswap_a
master[TOK_B] = master[TOK_B]*uniswap_b

In [ ]:
# rename To/From as User, drop all instances of Uniswap wallet address to make my life easier
for tok in [TOK_A, TOK_B]:
    lista = master[f'From{tok}']
    listb = master[f'To{tok}']
    listbool = master[f'From{tok}'].str.contains(PROTOCOL)
    master[f'User{tok}'] = [b if x else a for (x, a, b) in zip(listbool, lista, listb)]
    master.drop([f'From{tok}', f'To{tok}'], axis = 1, inplace = True)
master

In [ ]:
# replace router and nft with true end user if possible
master = replace_router(master)

In [ ]:
master = replace_nft(master)

In [ ]:
# reorder columns for my peace of mind
new_order = [0,1,2,3,7,9,10,4,5,6,8]
master = master[master.columns[new_order]]

In [ ]:
# Categorize transactions as Add/Swap/Remove
txns = uniswap_a + uniswap_b
if (1 in txns) or (-1 in txns):
    print('ugh')

master['Transaction Type'] = txns/2
master['Transaction Label'] = master['Transaction Type'].map({-1: 'Remove', 0: 'Swap', 1:'Add'})

Add Reference Rates (USD) from Coin Metrics

In [ ]:
pricesA = get_gecko_prices(TOK_A)
pricesA

In [ ]:
pricesB = get_gecko_prices(TOK_B)
pricesB

In [ ]:
prices = pricesA.copy()
prices['price'] = np.array(pricesA['price'])/np.array(pricesB['price'])
prices

Format prices to add into master dataframe

In [ ]:
market_price = np.zeros(len(master))

idx1 = 0
idx2 = 0

while idx1 < len(prices) and idx2 < len(market_price):
    #print('hi')
    ref = prices['time'].iloc[idx1]
    tx = master['timeStamp'].iloc[idx2]
    #print(ref)
    #print(tx)
    while ref.day >= tx.day and ref.hour > tx.hour:
        idx2 += 1
        tx = master['timeStamp'].iloc[idx2]
    #print(idx2)
    #print(tx)
        
    while ref.day == tx.day and ref.hour == tx.hour:
        #print(prices['time'].iloc[idx1], master['timeStamp'].iloc[idx2])
        market_price[idx2] = prices['price'].iloc[idx1]
        idx2 += 1
        if idx2 < len(market_price):
            tx = master['timeStamp'].iloc[idx2]
        else:
            break
    idx1 += 1

In [ ]:
master['Market Price'] = market_price
master

Identify bot transactions

In [ ]:
a = master[f'User{TOK_A}'].str.contains(MEV)
b = master[f'User{TOK_B}'].str.contains(MEV)
master['botFlag'] = a+b
print(f'There are {np.sum(a+b)} identified bot transactions.')

Save the master dataframe

In [ ]:
master_string = f'data/pool{POOL}_master_{start_block}_{end_block}.csv'
master.to_csv(master_string)

#### Or, load a pre-existing master CSV file

In [ ]:
master_string = 'data/pool3_master_18039179_18245998.csv'
# master_string = f'data/pool{POOL}_master_{start_block}_{end_block}.csv' # uncomment if loading a different CSV file
master = pd.read_csv(master_string)
master.drop('Unnamed: 0', axis = 1, inplace = True)
master['timeStamp'] = pd.to_datetime(master['timeStamp'])
master['Exchange Rate'].replace([np.inf, -np.inf], np.nan, inplace=True) # just in case
master

### Data Analysis

We calculate the effective exchange rate for swap transactions done in the pool.
- Negative exchange rate for swap means the user paid token A and got token B (negative flow of token B wrt Uniswap, so user gets tok B).
- Positive exchange rate for swap means the user paid token B and got token A.
- Infinite exchange rate is replaced by NaN

Split dataframe based on type of transaction.

In [ ]:
df_swap = master[master['Transaction Type'] == 0]
df_add = master[master['Transaction Type'] == 1] # To___ columns are irrelevant
df_remove = master[master['Transaction Type'] == -1] # From___ columns are irrelevant
print(f'There are {len(df_swap)} swaps, {len(df_add)} adds, and {len(df_remove)} removals.')

In [ ]:
data = np.array(df_swap[TOK_A][df_swap['botFlag'] == False])
mu = np.mean(data)
sig = np.std(data)
print(f'The data has mean {mu} and std. dev. {sig}.')

#### Fundamental (Market) Exchange Rate Versus Pool Exchange Rate

In [ ]:
plt.close()
plt.plot(df_swap['timeStamp'], np.abs(df_swap['Exchange Rate']), label = 'Pool Exchange Rate')
plt.plot(df_swap['timeStamp'], df_swap['Market Price'], label = 'Market Exchange Rate')
plt.legend()
plt.title(f'Real Exchange Rate Dynamics, {TOK_A}/{TOK_B} Pool with {FEE}% Fee')
plt.xlabel('Time')
plt.ylabel(f'{TOK_B} / {TOK_A}')
plt.show()

Looking at things from another direction, how often did swappers in the pool get a better rate than the CoinMetrics estimated hourly Reference Rate?

In [ ]:
# exchange rate negative, smaller absolute value better
pt1 = df_swap[df_swap['Exchange Rate'] < 0]
pt2 = pt1[-pt1['Exchange Rate'] < pt1['Market Price']]

In [ ]:
# exchange rate positive, bigger better
pt3 = df_swap[df_swap['Exchange Rate'] > 0]
pt4 = pt3[pt3['Exchange Rate'] > pt3['Market Price']]

In [ ]:
diff1 = pt2['Market Price'] + pt2['Exchange Rate']
diff2 = pt4['Exchange Rate'] - pt4['Market Price']
diff = np.concatenate([diff1, diff2])

In [ ]:
print(f"Out of {len(df_swap)} swap transactions, there were {len(pt2) + len(pt4)} with an exchange rate better than the estimated market reference rate.")
print(f"This means {round((len(pt2) + len(pt4))/len(df_swap)*100, 2)}% were advantageous swaps with a max improvement of {round(np.amax(diff),4)}.")

#### Gas Fees

In [ ]:
plt.close()
plt.scatter(np.abs(df_swap[TOK_B]), df_swap['gasFee']*df_swap['Market Price'], alpha = 0.1)
plt.show()

In [ ]:
# Average gas fee for swap
np.mean(df_swap['gasFee']*df_swap['Market Price'])

In [ ]:
plt.close()
plt.scatter(np.abs(df_add[TOK_B]), df_add['gasFee']*df_add['Market Price'], alpha = 0.1)
plt.show()

In [ ]:
# Average gas fee for liquidity addition
np.mean(df_add['gasFee']*df_add['Market Price'])

In [ ]:
plt.close()
plt.scatter(np.abs(df_remove[TOK_B]), df_remove['gasFee']*df_remove['Market Price'], alpha = 0.1)
plt.show()

In [ ]:
# Average gas fee for liquidity removal
np.mean(df_remove['gasFee']*df_remove['Market Price'])

#### Fitting Transactions

We examine the distribution of swap arrival times to determine whether it would be better to model swaps as a continuous process (most swaps occur at equally spaced intervals) or as a Poisson point process (random amounts of time between arrivals, time between arrivals would follow exponential distribution). 

In [ ]:
df_swap.drop(columns=['Transaction Type', 'Transaction Label'], inplace = True)
df_swap['Direction'] = np.sign(df_swap[TOK_B]) # +1 is swap USDC for TOK_A, -1 is swap TOK_A for USDC
df_swap = df_swap[np.abs(df_swap[TOK_B]) > 0.5]

In [ ]:
dt = (np.array(df_swap['timeStamp'].iloc[1:]) - np.array(df_swap['timeStamp'].iloc[:-1]))/np.timedelta64(1, 's')
df_swap['time diff'] = np.concatenate([np.zeros(1), dt])

In [ ]:
diff = np.abs(np.array(df_swap['Exchange Rate'])) - np.array(df_swap['Market Price']) # p^* - m^*
df_swap['price diff'] = diff#np.concatenate([np.zeros(1), diff])

In [ ]:
np.amin(df_swap['price diff'])

In [ ]:
df_swap['price diff'] = np.maximum(df_swap['price diff'], -200)

In [ ]:
exo_swap = df_swap[df_swap['botFlag'] == False]
exo_swap

In [ ]:
drop_list = []
for i in range(len(exo_swap)-2):
    if np.abs(exo_swap[TOK_B].iloc[i+2] + exo_swap[TOK_B].iloc[i]) < 50:
        drop_list.append(i)
        drop_list.append(i+2)
exo_extreme = exo_swap.drop(index=exo_swap.index[drop_list])
exo_extreme

##### Predicting transaction arrival times

In [ ]:
block_start = exo_swap['blockNumber'].iloc[0]
block_end = exo_swap['blockNumber'].iloc[-1]
tracker = np.zeros((block_end - block_start + 1, 2))

blk_ctr = 0
for i in range(len(exo_swap)):
    while exo_swap['blockNumber'].iloc[i] > blk_ctr + block_start:
        #print(f'Mismatch: {exo_swap["blockNumber"].iloc[i]} vs {block_start + blk_ctr}')
        tracker[blk_ctr,0] = -1
        tracker[blk_ctr, 1] = exo_swap['price diff'].iloc[i]
        blk_ctr += 1
    if exo_swap['blockNumber'].iloc[i] == blk_ctr + block_start:
        #print(f'{exo_swap["blockNumber"].iloc[i]} matches with {block_start + blk_ctr}')
        tracker[blk_ctr,0] = 1
        tracker[blk_ctr, 1] = exo_swap['price diff'].iloc[i]
        blk_ctr += 1
    #elif exo_swap['blockNumber'].iloc[i] < blk_ctr + block_start:
    #   print('repeated block in exo_swap')

In [ ]:
hist, bins = np.histogram(np.abs(tracker[:, 1]), bins = 20)
probs = np.zeros(len(hist))
for i in range(len(hist)):
    success_in_bin = np.sum((np.abs(tracker[:,1]) >= bins[i])*(np.abs(tracker[:,1]) < bins[i+1])*(tracker[:,0] == 1))
    probs[i] = success_in_bin/hist[i] # how many blocks are in bin and had a swap
    if probs[i] < 1e-2:
        print(i)
best_fit = np.polyfit(bins[:-1],probs, 1)

In [ ]:
m, b = best_fit
shift = -1 * math.log(1/b - 1)
scale = m/(b**2) * np.exp(shift)

In [ ]:
plt.close()
x = np.arange(np.amin(np.abs(tracker[:, 1])), np.amax(np.abs(tracker[:,1])))
y = 1/(1 + np.exp(-scale*x-shift))
plt.bar(bins[:-1], 1, edgecolor = 'C0', color = '#ededed', width = 1.5)
plt.bar(bins[:-1], probs, edgecolor = 'C0', width = 1.5)

plt.plot(x, y, label = 'Sigmoid')
plt.plot(x, np.poly1d(best_fit)(x), label = 'Linear')
plt.xlabel(r'$|p^* - m^*|$')
plt.ylabel('Probability of Swap Arrival')
plt.legend()
plt.show()

##### Predicting transaction direction and magnitude

A positive swap direction means that the user deposited Token B and received Token A, while a negative swap direction means the user deposited Token A and received Token B.

A positive $p^* - m^*$ value means that Token A is "worth more" in the pool, so users should want to deposit Token A in the pool and withdraw Token B (meaning a negative swap direction) while a negative $p^* - m^*$ means that Token A is cheaper in the pool, so users should want to exchange their Token B for Token A. 

**Note**: Swappers see the difference between the current market exchange rate and the pool rate after the previous transaction.

Investigation of size of swap vs size of $p^* - m^*$:

In [ ]:
plt.close()
offset = 1
data_x = exo_extreme['price diff'].iloc[:-offset] #[np.abs(exo_extreme[TOK_B]) > 1e6] 
x_reg = np.linspace(np.amin(data_x), np.amax(data_x), 50)
data_y = (np.sign(exo_extreme[TOK_B])*np.log(np.abs(exo_extreme[TOK_B]))).iloc[offset:] #exo_extreme[TOK_B].iloc[offset:] # [np.abs(exo_extreme[TOK_B]) > 1e6] #exo_extreme[TOK_B].iloc[offset:]
fit_result = stats.linregress(data_x, data_y)
m = fit_result[0]
b = fit_result[1]
print(f'p-value = {fit_result.pvalue} and R-value = {fit_result.rvalue}.') # p-test
plt.scatter(data_x, data_y, label = 'Data', alpha = 0.1)
plt.plot(x_reg, x_reg*m + b, color = 'red', label = f'y = {round(m, 2)}x + {round(b, 2)}')
plt.xlabel(r'$p^*-m^*$')
plt.ylabel(r'sgn$(\xi)\ \log_{10}(|\xi|)$')
plt.legend()
plt.show()

Now let us view the joint density of $p^*-m^*$ and signed $\log(\xi)$:

In [ ]:
from mpl_toolkits.mplot3d import axes3d

In [ ]:
arb_list = np.linspace(np.amin(data_x), np.amax(data_x), 100)
usdc_list = np.linspace(np.amin(data_y), np.amax(data_y), 100)
grid_x, grid_y = np.meshgrid(arb_list, usdc_list)
points = np.vstack([grid_x.ravel(), grid_y.ravel()])
points.shape

In [ ]:
input_data = np.vstack((np.array(data_x), np.array(data_y)))
input_data.shape

In [ ]:
mvkernel0 = stats.gaussian_kde(input_data, bw_method = 0.4/input_data.std(ddof=1)) # A 2-D array with shape (# of dims, # of data)

In [ ]:
mvdens0 = mvkernel0(points)

In [ ]:
plt.close()
ax = plt.figure().add_subplot(projection='3d')

# Plot the 3D surface
ax.plot_surface(grid_x, grid_y, mvdens0.reshape(grid_x.shape), edgecolor='black', lw=0.2, alpha=0.3)#rstride=8, cstride=8)

# Plot projections of the contours for each dimension.  By choosing offsets
# that match the appropriate axes limits, the projected contours will sit on
# the 'walls' of the graph.
#ax.contour(grid_x, grid_y, mvdens0.reshape(grid_x.shape), zdir='z', offset=0, cmap='coolwarm')
ax.contour(grid_x, grid_y, mvdens0.reshape(grid_x.shape), zdir='x', offset=arb_list[0])#, cmap='coolwarm')
ax.contour(grid_x, grid_y, mvdens0.reshape(grid_x.shape), zdir='y', offset=usdc_list[-1])#, cmap='coolwarm')

ax.set(xlim=(arb_list[0], arb_list[-1]), ylim=(usdc_list[0], usdc_list[-1]), zlim=(0, 0.02), xlabel=r'$p^*-m^*$', ylabel=r'sgn$(\xi)\ \log_{10}(|\xi|)$')

plt.show()

If you would like to see the conditional CDF for a given $p^*-m^*$:

In [ ]:
points2 = np.zeros((2, len(usdc_list)))
points2[0, :] = 4
points2[1, :] =usdc_list
cond_pdf= mvkernel0(points2)
cond_cdf = np.cumsum(cond_pdf)
cond_cdf = cond_cdf/cond_cdf[-1]

In [ ]:
plt.close()
plt.plot(usdc_list, cond_pdf)
plt.show()

How do we simulate points from this distribution?

In [ ]:
def inverse_finder(x, domain, cdf):
    i = np.searchsorted(cdf, x)
    #print(i, cdf[i-1], cdf[i], domain[i-1], domain[i])
    alpha = (x - cdf[i-1])/(cdf[i] - cdf[i-1])
    #print(alpha)
    sample = domain[i-1] + alpha*(domain[i]-domain[i-1])
    return sample

In [ ]:
inverse_finder(0.45, usdc_list, cond_cdf)

#### Investigate MEV bot behavior

Let us make a preliminary pass at identifying bot attacks in the data. The method is as follows:
1. Look at a transaction.
2. "Scan" the next-next for another transaction from the same user or a known pair of bot attackers.
3. If found, check whether the second transaction has the same size and opposite sign, within an error margin of 5%.
4. If so, these two transactions constitute a bot attack! Consolidate the whole "attack scheme" into one row of attacks.
5. Move on to the next transaction after the attack.

In [ ]:
attacks = pd.DataFrame({'User':[], TOK_A:[], TOK_B:[], 'Profit':[], 'Perc. Profit':[], 'Start':[], 'End':[], 'Idx1':[], 'Idx2':[], 'Txn Scale': []})
curr_df = master
i = -1
while i < len(curr_df)-3:
    i += 1
    user1 = [curr_df[f'User{TOK_A}'].iloc[i], curr_df[f'User{TOK_B}'].iloc[i]]
    user2 = [curr_df[f'User{TOK_A}'].iloc[i+2], curr_df[f'User{TOK_B}'].iloc[i+2]]
    
    users = user1 + user2
    user_overlap = list(set(user1) & set(user2))
    matchFlag = False
    
    '''
    # the user matching process is a little complicated because of certain bot "paired" behaviors observed in the data
    if len(user_overlap) >=1:
        user = user_overlap[0]
        matchFlag = True
    elif weird_bot_1 in users and weird_bot_2 in users: # weird bot combo matches
        user = weird_bot_1
        matchFlag = True
    
    '''
    # ok here's the dumb way:
    if user1[0] == user2[0] and user1[0] not in ROUTER: # first user matches
        user = user1[0]
        matchFlag = True
    elif user1[1] == user2[1] and user1[1] not in ROUTER: # second user matches
        user = user1[1]
        matchFlag = True
    elif weird_bot_1 in users and weird_bot_2 in users: # weird bot combo matches
        user = weird_bot_1
        matchFlag = True

    idx1 = curr_df.index[i]
    idx2 = curr_df.index[i+2]
    
    start = curr_df['timeStamp'].iloc[i]
    end = curr_df['timeStamp'].iloc[i+2]

    # if indices are close enough and users match, check transaction sizes
    if (idx2 - idx1 == 2) and (matchFlag == True) and ((end - start)//np.timedelta64(1, 's') == 0):
        pi_a = curr_df[TOK_A].iloc[i+2] + curr_df[TOK_A].iloc[i]
        pi_b = curr_df[TOK_B].iloc[i+2] + curr_df[TOK_B].iloc[i]
        scale = np.abs(curr_df[TOK_B].iloc[i]) # just the general size of the transaction

        err_a = np.abs(pi_a)/max(np.abs(curr_df[TOK_A].iloc[i]), 1e-4)
        err_b = np.abs(pi_b)/max(np.abs(curr_df[TOK_B].iloc[i]), 1e-4)
        
        flag1 = (curr_df['Transaction Type'].iloc[i] == 0) and (curr_df['Transaction Type'].iloc[i+2] == 0)
        flag2 = (curr_df['Transaction Type'].iloc[i] == 1) and (curr_df['Transaction Type'].iloc[i+2] == -1)

        if (flag1 or flag2) and (min(err_a, err_b) < 0.05):
            botflag = (user in MEV)
            
            if curr_df['Transaction Type'].iloc[i] != 0:
                attackType = 1 # liquidity-based attack
            else:
                attackType = 0 # swap-based attack

            gas = curr_df['gasFee'].iloc[i] + curr_df['gasFee'].iloc[i+2]

            curr_profit = -(pi_a*curr_df['Market Price'].iloc[i] + pi_b) - gas # recall that a negative transaction is going away from Uniswap, aka TO the user
            rel = curr_profit/scale

            attacks = attacks.append({'User':user, TOK_A:-pi_a, TOK_B:-pi_b, 'Profit':curr_profit, 'Perc. Profit': rel, 'Idx1':idx1, 'Idx2':idx2, 'Start':start, 'End':end, 'Txn Scale': scale, 'botFlag': botflag, 'Type': attackType}, ignore_index = True)

print(len(attacks))

What percentage of bot attacks are swap sandwich attacks vs Just-in-Time (JIT) liquidity attacks?

In [ ]:
liq_attacks = np.sum(attacks['Type'] == 1)
swap_attacks = np.sum(attacks['Type'] == 0)
if liq_attacks + swap_attacks != len(attacks):
    print('Issue')
else:
    print(f'Liquidity attacks were {round(liq_attacks/len(attacks), 4)*100}% of total.')

What percentage of identified bot attacks are done by our identified MEV bots?

In [ ]:
np.sum(attacks['botFlag'] == 1)/len(attacks)

To determine the value of $\zeta$, we must answer: what percentage of vulnerable transactions does the Bot attack?

In [ ]:
large_threshold = 1e5

In [ ]:
n1 = len(attacks) # number of bot attacks

# number of vulnerable transactions
n2 = 0
for i in range(len(df_swap)):
    big_enough = np.abs(df_swap[TOK_B].iloc[i]) > large_threshold
    right_direction = ((np.abs(df_swap['Exchange Rate'].iloc[i]) - df_swap['Market Price'].iloc[0])*np.sign(df_swap[TOK_B].iloc[0]) > 0)
    not_bot = ~df_swap['botFlag'].iloc[0]
    if big_enough and right_direction and not_bot:
        n2 += 1

print(f'The bot attack rate, zeta, is {round(n1/n2 * 100, 3)}%.')

#### Looking at LP Behavior in the Pool

In [ ]:
df_liq = master[master['Transaction Type'] != 0]
df_liq

First, here is some code to visualize swaps and liquidity magnitude and direction for a small chunk of transactions at whichever index I choose.

In [ ]:
i = 50
chunk = master.iloc[i:i+15]

tracker = 1
for idx in range(1, len(chunk)):
    if chunk['blockNumber'].iloc[idx] == chunk['blockNumber'].iloc[idx - tracker]:
        chunk['blockNumber'].iloc[idx] += tracker/10
        tracker +=1
        
    else:
        tracker = 1

df_bot = chunk[chunk['botFlag'] == True]
not_bot = chunk[chunk['botFlag'] == False]

swap_bot = df_bot[df_bot['Transaction Type'] == 0]
swap_not = not_bot[not_bot['Transaction Type'] == 0]

lp_bot = df_bot[df_bot['Transaction Type'] != 0]
lp_not = not_bot[not_bot['Transaction Type'] != 0]

In [ ]:
plt.close()

plt.yscale('symlog')

plt.stem(swap_not['blockNumber'], swap_not[TOK_A], linefmt = 'g', markerfmt = "gx", label = 'Swap, People')
plt.stem(lp_bot['blockNumber'], lp_bot[TOK_A], linefmt = 'r', markerfmt = "rx", label = 'Liquidity, Bots')
plt.stem(lp_not['blockNumber'], lp_not[TOK_A], linefmt = 'b', markerfmt = "bx", label = 'Liquidity, People')
plt.title(f'DEX Transactions')
plt.legend()#bbox_to_anchor=(1.05, 0.55))
plt.xlabel('Block Number')
plt.ylabel('Value (USDC)')
plt.axhline(0, color = 'black')
ax = plt.gca()
ax.get_xaxis().get_major_formatter().set_scientific(False)
ax.get_xaxis().get_major_formatter().set_useOffset(False)
plt.show()

In [ ]:
plt.close()

fig, axs = plt.subplots(2)

axs[0].set_yscale('symlog')
axs[1].set_yscale('symlog')

#axs[0].stem(swap_bot['timeStamp'], swap_bot[TOK_A], linefmt = 'k', markerfmt = "kx", label = 'Swap, Bots')
axs[0].stem(swap_not['blockNumber'], swap_not[TOK_A], linefmt = 'g', markerfmt = "gx", label = 'Swap, People')
axs[0].stem(lp_bot['blockNumber'], lp_bot[TOK_A], linefmt = 'r', markerfmt = "rx", label = 'Liquidity, Bots')
axs[0].stem(lp_not['blockNumber'], lp_not[TOK_A], linefmt = 'b', markerfmt = "bx", label = 'Liquidity, People')
axs[0].set_title(f'{TOK_A} Transactions')
axs[0].legend(bbox_to_anchor=(1.05, 0.55))

axs[1].stem(swap_bot.index, swap_bot[TOK_B], linefmt = 'k', markerfmt = "kx", label = 'Swap, Bots')
axs[1].stem(swap_not.index, swap_not[TOK_B], linefmt = 'g', markerfmt = "gx", label = 'Swap, People')
axs[1].stem(lp_bot.index, lp_bot[TOK_B], linefmt = 'r', markerfmt = "rx", label = 'Liquidity, Bots')
axs[1].stem(lp_not.index, lp_not[TOK_B], linefmt = 'b', markerfmt = "bx", label = 'Liquidity, People')
axs[1].set_title(f'{TOK_B} Transactions')

plt.show()

##### Value of Liquidity Contributions

In [ ]:
lps = df_liq[df_liq['botFlag'] == False]

In [ ]:
from scipy.signal import argrelextrema

In [ ]:
value = np.log10(np.array(np.abs(lps['USDC']) + np.abs(lps['ETH'])*np.array(lps['Market Price'])))
value = value[value > 1]
kde = KernelDensity(kernel='gaussian', bandwidth=0.25).fit(value[:, np.newaxis])
s = np.linspace(0, np.amax(value), 100)
e = np.exp(kde.score_samples(s.reshape(-1,1)))

mi, ma = argrelextrema(e, np.less)[0], argrelextrema(e, np.greater)[0]
print("Minima:", s[mi], mi)
print("Maxima:", s[ma], ma)

In [ ]:
plt.close()

plt.fill_between(s[:mi[0]+1], 0, e[:mi[0]+1], color='C0')
for i in range(len(mi)-1):
    plt.fill_between(s[mi[i]:mi[i+1]+1], 0, e[mi[i]:mi[i+1]+1], color=f'C{i+1}')
    plt.vlines(s[mi[i]], 0, e[mi[i]], color = 'white')
plt.vlines(s[mi[-1]], 0, e[mi[-1]], color = 'white')
plt.fill_between(s[mi[-1]:], 0, e[mi[-1]:], color=f'C{len(mi)}')

plt.scatter(s[ma], e[ma], color = 'black')

plt.xlabel('Log Liquidity Contribution')
plt.ylabel('Density')
plt.show()

In [ ]:
capital_list = 10**s[ma]
capital_list

In [ ]:
total = np.trapz(e, s)
small = np.trapz(e[:mi[0]], s[:mi[0]])
mid = np.trapz(e[mi[0]:mi[1]], s[mi[0]:mi[1]])
large = np.trapz(e[mi[1]:], s[mi[1]:])
probs = np.array([small, mid, large])/total
print(probs)

Let's check the estimated density vs the actual data:

In [ ]:
len(value[(value >= s[mi[0]])*(value < s[mi[1]])])/len(value)

In [ ]:
len(value[value < s[mi[0]]])/len(value)

In [ ]:
len(value[value > s[mi[1]]])/len(value)

##### How long do LPs leave money in pool?

Identify additions and removals done by the same address:

In [ ]:
# Match transactions in df_add and df_remove that came from the same user account
user_list_add = list(df_add[f'User{TOK_A}'])
user_list_add = user_list_add + list(df_add[f'User{TOK_B}'])
lp_add = [*set(user_list_add)]

user_list_rem = list(df_remove[f'User{TOK_A}'])
user_list_rem = user_list_rem + list(df_remove[f'User{TOK_B}'])
lp_rem = [*set(user_list_rem)]

print(len(lp_add))
print(len(lp_rem))

In [ ]:
common = list(set(user_list_add).intersection(user_list_rem))
print(len(common)) # common usernames between two lists

In [ ]:
in_out = pd.DataFrame(common)
in_out.rename(columns={0:'User'}, inplace = True)

In [ ]:
add_dates = []
rem_dates = []
for user in common:
    add_dates.append(np.array(df_add.index[df_add[f'User{TOK_A}'].str.contains(user) + df_add[f'User{TOK_B}'].str.contains(user)]))
    rem_dates.append(np.array(df_remove.index[df_remove[f'User{TOK_A}'].str.contains(user)+ df_remove[f'User{TOK_B}'].str.contains(user)]))
in_out['Add Idx'] = add_dates
in_out['Remove Idx'] = rem_dates

In [ ]:
positions = pd.DataFrame(columns=['user', 'add_idx', 'remove_idx', 'Value', 'Profit', 'Time'])
for i in range(len(common)):
    liq = []
    for idx in in_out['Add Idx'][i]:
        for j in in_out['Remove Idx'][i]:
            if j > idx:
                start_a = df_add[TOK_A][idx]
                start_b = df_add[TOK_B][idx]
                add_date = df_add['timeStamp'][idx]
                
                end_a = -df_remove[TOK_A][j]
                end_b = -df_remove[TOK_B][j]
                rem_date = df_remove['timeStamp'][j]
                
                dA = (end_a - start_a)
                dB = (end_b - start_b)
                
                val = end_a + end_b * df_remove['Market Price'][j]
                pi = dA + dB * df_remove['Market Price'][j]
                
                df2 = {'user': in_out['User'][i], 'add_idx':idx, 'remove_idx':j, 'Value':val , 'Profit': pi, 'Time':(rem_date - add_date)/np.timedelta64(1, 's')}
                positions = positions.append(df2, ignore_index = True)
                break
positions

In [ ]:
# average time that liquidity is left in pool before adjustment
np.mean(positions['Time'])/3600 # units are hours

In [ ]:
#exclude JIT liquidity adjustments
np.mean(positions['Time'][positions['Time'] > 0])/3600 

In [ ]:
plt.close()
plt.scatter(positions['Value'], positions['Time']/3600)
plt.xlabel('Value of Liquidity Position Upon Removal')
plt.ylabel('Length of Time in Pool (hrs)')
plt.show()

##### Position Width

In [ ]:
def tick_to_price(tick, dec_a, dec_b):
    # dec_a is the decimals for token a, dec_b is the decimals for token b numbers are stored by shifting decimal point to reduce rounding errors)
    pt1 = 1.0001 ** tick
    pt2 = pt1 * (10 ** (dec_b - dec_a)) # this gives prices as tokA/tokB
    pt3 = 1/pt2 # we usually want tokB/tokA i.e. USDC/ETH for Market Price
    
    return pt3

In [ ]:
def get_position(contract, start_block, end_block):
    df = pd.DataFrame(columns=['Txnhash', 'Owner', 'amount0', 'amount1', 'Position', 'Width'])
    
    mint_topic = '0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde'
    url1=f'https://api.etherscan.io/api?module=logs&action=getLogs&address={contract}&fromBlock={start_block}&toBlock={end_block}&topic0={mint_topic}&page=1&offset=1000&apikey={API_KEY}'
    response = requests.get(url1)
    resp = response.json()
    
    n_txn = len(resp['result'])
    print(n_txn)
    
    for i in range(n_txn):
        # parse JSON and remove 0x in front of hex values
        accountHex = resp['result'][i]['topics'][1][2:]
        lowerHex = resp['result'][i]['topics'][2][2:]
        upperHex = resp['result'][i]['topics'][3][2:]
        data = resp['result'][i]['data'][2:]

        # convert from ABI to readable numbers
        owner = eth_abi.decode(['address'], bytes.fromhex(accountHex), strict = False)[0]
        lowerTick = eth_abi.decode(['int24'], bytes.fromhex(lowerHex), strict = False)[0]
        upperTick = eth_abi.decode(['int24'], bytes.fromhex(upperHex), strict = False)[0]
        sender, amount, amount0, amount1 = eth_abi.decode(['address', 'uint128', 'uint256', 'uint256'], bytes.fromhex(data), strict = False)

        amount0 = amount0 * 1e-6 # USDC off by 10^{-6}
        amount1 = amount1 * 1e-18 # ETH off by 10^{-18}
        
        df2 = {'Txnhash': resp['result'][i]['transactionHash'], 'Owner': owner, 'amount0': amount0, 'amount1': amount1, 'Position': np.array([lowerTick, upperTick]), 'Width': (upperTick - lowerTick)}
        df = df.append(df2, ignore_index = True)
    
    return df

In [ ]:
liq_df = get_position(PROTOCOL, start_block, end_block)

In [ ]:
tokA_price = np.mean(master['Market Price']) # could be more accurate
liq_df['Value'] = np.array(liq_df['amount0']) + np.array(liq_df['amount1']) * tokA_price

In [ ]:
# this bot makes many many transactions
bigMEV = weird_bot_1
bot_liq = liq_df[liq_df['Owner'] == bigMEV]
print(np.mean(bot_liq['Width']))
print(len(bot_liq))

In [ ]:
max_tick = 887270
min_tick = -887270

In [ ]:
tick_list = np.sort(np.vstack(liq_df['Position']).flatten())[13:-13] # get rid of extremes

In [ ]:
normal_liq = liq_df[(liq_df['Owner'] != bigMEV)*(liq_df['Width'] < (max_tick - min_tick))]

In [ ]:
plt.close()
plt.scatter(normal_liq['Value'], np.array(normal_liq['Width'], dtype = int))
plt.xlabel('Position Value')
plt.ylabel('Position Width')
#plt.plot(np.arange(1, 5e6, 10), 5e4/np.arange(1, 5e6, 10) + 10, color = 'black')
#plt.plot(np.arange(1, 5e6, 10), 1e5/np.exp(np.arange(1, 5e6, 10)) + 10, color = 'black')
plt.show()

In [ ]:
pi_LP_fee, pi_LP_liq, pi_B = bot.calcProfit(x_1, x_2, xi, nu)
pi_LP_fee, pi_LP_liq, pi_B

In [ ]:
print(f'Pool ER is {nu.pool_er} and Market ER is {nu.mkt_er}.')

In [ ]:
plt.close()
fig, ax = plt.subplots(2)
space = nu.size//20

ax[0].bar(np.arange(nu.size), nu.tokA, align = 'edge', tick_label = nu.ticks[1:], width = -1)
#ax[0].axvline(nu.idx, color = 'red')
ax[0].set_xticks(np.arange(-1, nu.size, space))
ax[0].set_xticklabels(np.round(nu.ticks[0::space],2), rotation = 45, ha = 'right')

ax[0].set_title(f'Token A Liquidity After Actual Swap')
ax[0].set_xlabel('Exchange Rate')
ax[0].set_ylabel('Quantity')

ax[1].bar(np.arange(nu.size), nu.tokB, align = 'edge', tick_label = nu.ticks[1:], width = -1)
#ax[1].axvline(nu.idx, color = 'red')
ax[1].set_xticks(np.arange(-1, nu.size, space))
ax[1].set_xticklabels(np.round(nu.ticks[0::space],2), rotation = 45, ha = 'right')

ax[1].set_title(f'Token B Liquidity After Actual Swap')
ax[1].set_xlabel('Exchange Rate')
ax[1].set_ylabel('Quantity')

plt.show()